In [48]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv
import requests
import os

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [49]:
client : OpenAI = OpenAI()

In [50]:
key  = os.environ["FMP_API_KEY"]
def income_statement(period):
    Url = f"https://financialmodelingprep.com/api/v3/income-statement/0000320193?period={period}&apikey={key}"
    res = requests.get(Url)
    return json.dumps(res.json())
def cash_flow_statement(period):
    Url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/0000320193?period={period}&apikey={key}"
    res = requests.get(Url)
    return json.dumps(res.json())

In [51]:
import json
from openai.types.beta import Assistant

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

assistant: Assistant = client.beta.assistants.create(
  instructions="You are a financial assistant specialist, excel in leveraging cutting-edge technology and data-driven insights to empower individuals and businesses in achieving their financial goals with precision and efficiency.",
  model="gpt-3.5-turbo-1106",
  tools=[{
      "type": "function",
    "function": {
      "name": "income_statement",
      "description": "Get the income statement quarter or annual as per parameter or desire query",
      "parameters": {
        "type": "object",
        "properties": {
          "period": {"type": "string", "description": "income statement quarter or annual"},
        },
        "required": ["period"]
      }
    }
  },{
      "type": "function",
    "function": {
      "name": "cash-flow-statement",
      "description": "Get the income statement quarter or annual as per parameter or desire query",
      "parameters": {
        "type": "object",
        "properties": {
          "period": {"type": "string", "description": "cash-flow-statement quarter or annual"},
        },
        "required": ["period"]
      }
    }
  }
  ])

In [52]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_4LBoDHaqCKK5ijOlfgTZjiaV', created_at=1708954614, metadata={}, object='thread')


In [53]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role = "user",
    content="Kindly show annual income statement and annually cash flow statement"
)
dict(message)

{'id': 'msg_8rAVLWRAx7OnbNzh9lqIsqIQ',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='Kindly show annual income statement and annually cash flow statement'), type='text')],
 'created_at': 1708954614,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_4LBoDHaqCKK5ijOlfgTZjiaV'}

In [54]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
dict(run)

{'id': 'run_pZ4fj9vjX7DHAVwpHRmnsnZX',
 'assistant_id': 'asst_JItrjGxn75vf1yoqWyBWDzby',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1708954615,
 'expires_at': 1708955215,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a financial assistant specialist, excel in leveraging cutting-edge technology and data-driven insights to empower individuals and businesses in achieving their financial goals with precision and efficiency.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_4LBoDHaqCKK5ijOlfgTZjiaV',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='income_statement', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'income statement quarter or annual'}}, 'required': ['period']}, description='Get the income statement quarter or annual as per parameter or desire

In [55]:
available_functions = {
    "income_statement":income_statement,
    "cash-flow-statement":cash_flow_statement
}

In [56]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    # show_json("Run Steps:", run_steps)
    print(runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'.....')
        print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    response = function_to_call(**function_args )
                        
                        
                    tool_outputs.append({
                        "tool_call_id": toolcall.id,
                        "output": response
                              })
                   
                   
            print(tool_outputs,">>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages= client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break

in_progress .....
Run is queued. Waiting...
requires_action .....
requires_action .....
Status:  requires_action


'submit_tool_outputs'

{'submit_tool_outputs': {'tool_calls': [{'id': 'call_2dTLGBekgh9YgUIsayxgaB6m',
    'function': {'arguments': '{"period": "annual"}',
     'name': 'income_statement'},
    'type': 'function'},
   {'id': 'call_4OEqtdEC7IDeX9asuEgyMkNo',
    'function': {'arguments': '{"period": "annual"}',
     'name': 'cash-flow-statement'},
    'type': 'function'}]},
 'type': 'submit_tool_outputs'}

toolCalls present:
[{'tool_call_id': 'call_2dTLGBekgh9YgUIsayxgaB6m', 'output': '[{"date": "2023-09-30", "symbol": "AAPL", "reportedCurrency": "USD", "cik": "0000320193", "fillingDate": "2023-11-03", "acceptedDate": "2023-11-02 18:08:27", "calendarYear": "2023", "period": "FY", "revenue": 383285000000, "costOfRevenue": 214137000000, "grossProfit": 169148000000, "grossProfitRatio": 0.4413112958, "researchAndDevelopmentExpenses": 29915000000, "generalAndAdministrativeExpenses": 0, "sellingAndMarketingExpenses": 0, "sellingGeneralAndAdministrativeExpenses": 24932000000, "otherExpenses": -565000000, "operatingExpenses": 54847000000, "costAndExpenses": 268984000000, "interestIncome": 3750000000, "interestExpense": 3933000000, "depreciationAndAmortization": 11519000000, "ebitda": 125820000000, "ebitdaratio": 0.3282674772, "operatingIncome": 114301000000, "operatingIncomeRatio": 0.2982141227, "totalOtherIncomeExpensesNet": -565000000, "incomeBeforeTax": 113736000000, "incomeBeforeTaxRatio": 0